In [101]:
import json
from copy import copy
import numpy
import torch

In [115]:
f = open('exp/json/iaaa.json')
recording = json.loads(f.readline())
# Removing multiple-speaker segments
recording['segments'] = [segment for segment in recording['segments'] if len(segment['speakers']) == 1]
# Removing extra speakers
recording['segments'] = [segment for segment in recording['segments'] if segment['speakers'][0]['speaker_id'] in ['A', 'B']]
# Dealing with multiple-vector segments
new_segments = []
for segment in recording['segments']:
    for i in range(len(segment['ivectors'])):
        new_segment = copy(segment)
        new_segment['ivectors'] = [segment['ivectors'][i]]
        new_segment['xvectors'] = [segment['xvectors'][i]]
        new_segments.append(new_segment)
recording['segments'] = new_segments
# Lists to numpy arrays
for segment in recording['segments']:
    segment['ivectors'][0]['value'] = numpy.asarray(segment['ivectors'][0]['value'])
    segment['xvectors'][0]['value'] = numpy.asarray(segment['xvectors'][0]['value'])
# Getting speakers list
speakers = list(set([segment['speakers'][0]['speaker_id'] for segment in recording['segments']]))
# Getting speakers oracle models
oracle_limit = 5
oracle_segments = [[segment for segment in recording['segments'] if segment['speakers'][0]['speaker_id'] == speaker][:oracle_limit] for speaker in speakers]
oracle_models = []
for n, speaker_n in enumerate(speakers):
    ivector_n = oracle_segments[n][0]['ivectors'][0]['value']
    xvector_n = oracle_segments[n][0]['xvectors'][0]['value']
    for segment in oracle_segments[n][1:]:
        ivector_n = ivector_n + segment['ivectors'][0]['value']
        xvector_n = xvector_n + segment['xvectors'][0]['value']
    ivector_n = ivector_n / len(oracle_segments[n])
    xvector_n = xvector_n / len(oracle_segments[n])
    oracle_models.append({ 'speaker_id': speaker_n, 'ivector': ivector_n, 'xvector': xvector_n })
print(oracle_models)

[{'speaker_id': 'B', 'ivector': array([ 0.20623618,  2.8614498 , -0.27671398,  0.39160178,  0.38859158,
        0.65683646,  1.7036226 ,  0.57393912, -1.5821079 ,  0.9477645 ,
        2.5796364 ,  1.876193  ,  0.03561908,  0.70905214,  1.7808874 ,
        1.00420013, -0.18094709,  1.21929856, -0.770906  ,  1.48952894,
        1.08831344,  0.59702882,  0.39772942, -0.18888009,  0.65102376,
       -1.04152562, -0.88492176,  0.5104168 , -1.08025063, -0.75617084,
        1.6634748 ,  0.3101024 ,  1.80501784, -0.98426256, -0.65429521,
        0.887347  , -0.61813259, -1.34029678,  1.57020478, -0.42798788,
        0.65978958,  1.07047666, -0.46155173,  0.13441003,  2.2525774 ,
       -0.05966534,  0.2770185 , -0.3646315 ,  0.4974825 , -0.30980389,
        0.3379844 ,  0.23525082,  1.8946598 , -0.16858158, -0.03372748,
       -0.52756918, -0.18286396, -0.60631342,  0.01256376, -1.09768828,
        0.33398596,  0.36326344,  0.21998428,  1.58361402, -0.88245638,
        0.78359514,  0.48864956,